# Интерфейсы

Интерфейс - контракт, по которому класс, его реализующий, предоставляет какие-то методы.

Использование интерфейсов сильно увеличи

# Обобщения

# Методы расширения

Функциональность любого типа можно расширить при помощи механизма методов расширения

In [14]:
// Так он должен, по-хорошему, объявляться.
public static class StringExtentions
{
    public static string AddBorders(this string str, char symbol='=', int size=3)
    {
        string border = new string(symbol, size);
        return border + str + border;
    }
}

Unhandled Exception: (3,26): error CS1109: Методы расширения должны быть определены в статическом классе верхнего уровня; StringExtentions является вложенным классом.

**Ввиду особенностей Jupyter Notebook (он оборачивает всё в класс) я объявляю метод расширения прямо в ячейке. В реальности требуется публичный статический класс**

In [15]:
//public static class StringExtentions
//{
    public static string AddBorders(this string str, char symbol='=', int size=3)
    {
        string border = new string(symbol, size);
        return border + str + border;
    }
//}

In [16]:
string str = "Hello";
str.AddBorders()

===Hello===

# LINQ

# Перечислимые типы (enums)

## 1. Базовые возможности

In [17]:
enum Color
{
    White,  // 0
    Red,    // 1
    Green,  // 2
    Blue,   // 3
    Orange, // 4
}

In [18]:
Color white = Color.White;  
Console.WriteLine(white);   // White

Color red = (Color)1;       // Так можно приводить к типу перечисления 
Console.WriteLine(red);     // Red

Color unknown = (Color)42;  // Нет ошибки!
Console.WriteLine(unknown); // 42

White
Red
42


In [19]:
Color green = Enum.Parse<Color>("Green");
green.ToString()

Green

In [20]:
Enum.TryParse<Color>("Blue", out Color blue);
blue.ToString()

Blue

In [21]:
// Посмотрим, какими типами можно задавать перечисления
enum Dummy : char {}

Unhandled Exception: (2,14): error CS1008: Требуется тип byte, sbyte, short, ushort, int, uint, long или ulong.

## 2. Приведение перечислимых типов

In [22]:
enum Fruit
{
    Melon,      // 0
    Tomato,     // 1
    Apple,      // 2
    Blueberry,  // 3
    Orange,     // 4
}

In [23]:
Fruit orange = Color.Orange; // Безопасность типов -> ошибка

Unhandled Exception: (1,16): error CS0266: Не удается неявно преобразовать тип "Color" в "Fruit". Существует явное преобразование (возможно, пропущено приведение типов).

In [24]:
Fruit tomato = (Fruit)Color.Red; // А вот так уже можно
Console.WriteLine(tomato);

Tomato


In [25]:
Color unknownColor = (Color)42;
Fruit unknownFruit = (Fruit)unknownColor;
Console.WriteLine(unknownFruit);

42


In [26]:
// Любой enum имеет следующую цепочку наследования: MyEnum <- System.Enum <- System.ValueType <- System.Object

Enum enumEnum = Color.Blue;
ValueType enumValueType = Color.Blue;
object enumObj = Color.Blue;          // BOXING

Console.WriteLine($"{enumEnum}, {enumValueType}, {enumObj}");

Blue, Blue, Blue


## 3. Использование одного целочисленного значения для нескольких enum значений

In [42]:
public enum Subject
{
    Programming = 0,
    DiscreteMath = 1,
    Algebra = 2,
    Calculus = 3,
    Economics = 4,

    HardestSubject = Algebra,
    MostUsefulSubject = Programming,
    // HatefulSubject = Programming
}

In [28]:
Console.WriteLine(Subject.Programming);
Console.WriteLine(Subject.MostUsefulSubject);
Console.WriteLine((Subject)0);

Console.WriteLine(Subject.Programming == Subject.MostUsefulSubject)

Programming
Programming
Programming
True


In [29]:
Console.WriteLine(Subject.Algebra);
Console.WriteLine(Subject.HardestSubject);
Console.WriteLine((Subject)2);

Console.WriteLine(Subject.Algebra == Subject.HardestSubject)

Algebra
Algebra
Algebra
True


## 4. Рефлексия перечислимых типов

Статический метод Enum.GetUnderlyingType возвращает целочисленный тип для енама

In [30]:
Enum.GetUnderlyingType(typeof(Subject))

System.Int32

In [31]:
Enum.GetUnderlyingType(typeof(Subject))

System.Int32

В типе System.Type также есть метод GetEnumUnderlyingType

In [32]:
typeof(Subject).GetEnumUnderlyingType()

System.Int32

Который работает только с объектами-типами енамов

In [33]:
typeof(short).GetEnumUnderlyingType()

Unhandled Exception: Type provided must be an Enum. (Parameter 'enumType')

Можно получить все значения енама c помощью Enum.GetValues(Type)

In [34]:
var enumValues = Enum.GetValues(typeof(Subject)); // Аналог: typeof(Subject).GetEnumValues();
foreach(var value in enumValues){
    Console.WriteLine(value);
}

Programming
Programming
DiscreteMath
Algebra
Algebra
Calculus
Economics


In [35]:
Enum.GetNames(typeof(Subject)) // Аналог: typeof(Subject).GetEnumNames()

index,value
0,MostUsefulSubject
1,Programming
2,DiscreteMath
3,Algebra
4,HardestSubject
5,Calculus
6,Economics


In [36]:
Enum.IsDefined(typeof(Subject), 3)

True

In [37]:
Enum.IsDefined(typeof(Subject), 42)

False

## 5. Битовые флаги

In [38]:
[Flags]
enum FilePermission : byte
{
    None    = 0b00000000,

    Read    = 0b00000001,
    Write   = 0b00000010,
    Execute = 0b00000100,
    Rename  = 0b00001000,
    Move    = 0b00010000,
    Delete  = 0b00100000,

    User        = Read | Execute,
    ReadWrite   = Read | Write,
    Admin       = Read | Write | Execute | Rename | Move | Delete
}

[Про FlagsAttributes](https://docs.microsoft.com/ru-ru/dotnet/api/system.flagsattribute?view=net-5.0)

In [39]:
FilePermission permission = FilePermission.User;
permission.HasFlag(FilePermission.Read)

True

Пример использования:
```
void RenameFile(File file, User user)
{
    if (!user.Permission.HasFlag(FilePermission.Rename)) {
        throw new SomeException("you can't.")
    }
    ...
}
```

In [40]:
for (int i = 0; i <= 16; ++i) {
    FilePermission fp = (FilePermission)i;
    Console.WriteLine(fp.ToString("G"));
}

None
Read
Write
ReadWrite
Execute
User
Write, Execute
Write, User
Rename
Read, Rename
Write, Rename
ReadWrite, Rename
Execute, Rename
User, Rename
Write, Execute, Rename
Write, User, Rename
Move


### Пример: System.AttributeTargets

In [ ]:
[Flags, Serializable]
public enum AttributeTargets {
    Assembly = 0x0001,
    Module = 0x0002,
    Class = 0x0004,
    Struct = 0x0008,
    Enum = 0x0010,
    Constructor = 0x0020,
    Method = 0x0040,
    Property = 0x0080,
    Field = 0x0100,
    Event = 0x0200,
    Interface = 0x0400,
    Parameter = 0x0800,
    Delegate = 0x1000,
    ReturnValue = 0x2000,
    GenericParameter = 0x4000,
    All = Assembly | Module | Class | Struct | Enum |
    Constructor | Method | Property | Field | Event |
    Interface | Parameter | Delegate | ReturnValue |
    GenericParameter
}

## 6. Методы расширения для enum

Перечислениям можно "добавлять функциональность" с помощью методов расширения

In [43]:
//public static class EnumExtentions
//{
    public static int GetMark(this Subject subject)
    {
        return subject switch 
        {
            Subject.Programming => 8,
            Subject.DiscreteMath => 10,
            Subject.Algebra => 5,
            Subject.Calculus => 7,
            Subject.Economics => 6,
            _ => 0,
        };
    }
//}

In [44]:
Subject prog = Subject.Programming;
prog.GetMark()

8

# Атрибуты

In [ ]:
[Serializable][Flags]
[Serializable, Flags]
[FlagsAttribute, SerializableAttribute]
[FlagsAttribute()][Serializable()]

### Дублировать атрибуты нельзя

In [2]:
[Flags][Flags]
public enum MyFlags 
{
    Hello = 1,
    World = 2
}

Unhandled Exception: (1,9): error CS0579: Повторяющийся атрибут "Flags"

In [ ]:
Определяя конструктор экземпляров класса атрибутов, а также поля и свойства, 
следует ограничиться небольшим набором типов данных. Допустимы типы: Boolean, Char, Byte, SByte, Int16, UInt16, Int32, UInt32, Int64, UInt64, Single, Double, 
String, Type, Object и перечислимые типы.

Обнаружив настраиваемый атрибут, компилятор создает экземпляр класса 
этого атрибута, передавая его конструктору все указанные параметры. Затем он 
присваивает значения открытым полям и свойствам, используя для этого усовершенствованный синтаксис конструктора. Инициализировав объект настраиваемого 
атрибута, компилятор сериализует его и сохраняет в таблице метаданных